Business Plan Checker

Upload startup pitch (PDF), query = “What risks are identified?”, output = 
summarized risk section.

PyPDF2==extracts text from PDF.
openai==is used to call the model.
ipywidgets==provides a file uploader widget inside Jupyter.

tiktoken==helps with token counting (useful when chunking).

for taking the usesr input

Extract text from uploaded file

RAG approach
for asking the question and to get the output

In [1]:
!pip install streamlit 


In [2]:
import os
import streamlit as st
import PyPDF2
from google import genai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

api_key = os.getenv("google_Gemini_key")
#genai.configure(api_key=api_key)
client = genai.Client(api_key=api_key)

In [3]:
# Extract text

def extract_pdf_text(uploaded_file):
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [4]:
# Ask Gemini on each chunk

def ask_gemini(chunk, question):
    prompt = f"""
You are a startup pitch deck analyst.
Answer the user's question only from this chunk.

Question: {question}

Chunk:
\"\"\"{chunk}\"\"\"
If nothing relevant found, reply: "No relevant info in this chunk."
"""
    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
        contents=prompt
    )
    return response.text

In [5]:
#Synthesize final response

def combine_answers(answers):
    prompt = f"""
You are a startup expert. Combine the following extracted answers
from multiple PDF chunks into one final clean answer.

Extracted answers:
{answers}

Provide a structured, clear final answer.
"""

    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
         contents=prompt
    )
    return response.text

In [6]:
## Streamlit UI
# --------------------------
st.title("Business Plan Checker")
st.write("Upload a pdf and ask related question.")

uploaded_file = st.file_uploader("Upload PDF", type=["pdf"])
question = st.text_input("Ask a question about the PDF:")

if st.button("Get Answer"):
    if not uploaded_file:
        st.error("Please upload a PDF.")
    elif not question.strip():
        st.error("Please enter a question.")
    else:
        with st.spinner("Reading PDF..."):
            pdf_text = extract_pdf_text(uploaded_file)
            chunks = split_into_chunks(pdf_text)
            st.success(f"PDF Loaded! Total chunks: {len(chunks)}")

        #st.write("🔍 **Processing chunks...**")

        chunk_answers = []
        for i, c in enumerate(chunks):
            st.write(f"Analyzing {i+1}/{len(chunks)}...")
            ans = ask_gemini(c, question)
            chunk_answers.append(ans)

        final_answer = combine_answers("\n".join(chunk_answers))

        st.subheader(" Answer")
        st.write(final_answer)

2025-11-30 22:33:53.104 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.560 
  command:

    streamlit run c:\Users\Meghanshi Mathur\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-30 22:33:53.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-30 22:33:53.569 Thread 'Main